<a href="https://colab.research.google.com/github/rilschultz/CS167Notes/blob/main/Day24_RNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS167: Day24
## Recurrent Neural Networks

#### CS167: Machine Learning, Spring 2023

Tuesday, April 25th, 2023

📆 [Course Schedule](https://docs.google.com/spreadsheets/d/e/2PACX-1vSvFV5Mz0_YZE1d5r3gQ8IMktE4cBAsJIlP30cl2GhEpSO0J-YWV62QokSDz-OcOCsEmxMuKpY0kVlR/pubhtml?gid=0&single=true) | 🙋[PollEverywhere](https://pollev.com/meredithmoore011) | 📜 [Syllabus](https://analytics.drake.edu/~moore/cs167_s23_syllabus.html) | 📬 [CodePost Login](https://codepost.io/login)

# Admin Stuff

You should be working on:
- Notebook #6 is due tonight by 11:59pm
- [Project #2](https://classroom.github.com/a/gjJKXa78) is released today, due 5/5 by 11:59
- Quiz #3 will be release on Tuesday 5/2 and will be due on Tuesday, 5/9

## Load your data:

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Quick Review:

# Pulling it all together:

Okay, so we've talked about __image data__, __dot product__, __convolutions__, __max pooling__, and I've promised that we'll be able to pull off some computer vision. Let's connect the dots.

Things to notice:
- input is an image (in this case a color image, so 3 channels--red,green, and blue)
- There are several filters, not just one.
- `Conv2D` layers with `ReLU` are often followed by `maxpool` 
- Towards the end of the model, we switch to fully connected (`Dense`) layer
- We have as many output nodes as we have classes to predict.

<div>
<img src="https://github.com/merriekay/S23-CS167-Notes/blob/main/images/day22_cnn_architecture.svg?raw=1" width=800/>
</div>

[[img src](https://developers.google.com/machine-learning/practica/image-classification)]

## 🙋 PollEverywhere 

# ✨ New Material

# Resources for Today:

- [Intro to RNNs](https://towardsdatascience.com/a-brief-introduction-to-recurrent-neural-networks-638f64a61ff4)
- [Unreasonable Effectiveness of RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness)

# Types of Data:

Which of the following can we model with what we've learned so far? Why or why not?
- Predict the price of an airline ticket given the date, location, and weather
- Predict the percent chance that a customer will churn (cancel whatever service a company provides) given their usage data.
- Translate a phrase from one langauge to another
- Predict the next word in a sentence
- Predict if an image has a dog in it
- Predict what was said in an audio recording
- Generate text to caption an image
- Predict what the price of a stock will be next week

# So far our modeling has been limited:

We've only worked with a certain type of data:
- tabular data
- no timeseries (the data does not need to be in a specific order)
- the kind of predictions we have made have either been __classifications__ (binary or multiclass) or __regressions__.

Can you think of other kinds of target variables we might be interested in modeling?

# What if my input isn't the same size of my output?
<div>
<img src="https://github.com/merriekay/S23-CS167-Notes/blob/main/images/day23_input_output.png?raw=1" width=1000/>
</div>

## One to One:

This is referred to as a Vanilla Neural Network, we input one training example and make one prediction.

<div>
<img src="https://github.com/merriekay/S23-CS167-Notes/blob/main/images/day23_one2one.png?raw=1" width=100/>
</div>

## One to Many:

Can anyone think of an example of a ML model that is __one to many__?
- we input one training example, and output many predictions

<div>
<img src="https://github.com/merriekay/S23-CS167-Notes/blob/main/images/day23_one2many.png?raw=1" width=200/>
</div>

Image captioning:

<div>
<img src="https://github.com/merriekay/S23-CS167-Notes/blob/main/images/day23_image_caption.png?raw=1" width=800/>
</div>

## Many to One:

Can anyone think of an example of a ML model that is __many to one__?
- we input multiple things, and make one prediction from it

<div>
<img src="https://github.com/merriekay/S23-CS167-Notes/blob/main/images/day23_many2one.png?raw=1" width=200/>
</div>

Predicting a rating from a movie reivew

<div>
<img src="https://github.com/merriekay/S23-CS167-Notes/blob/main/images/day23_rating.png?raw=1" width=800/>
</div>

## Many to Many: 

Can anyone think of an example of a ML model that is __many to many__?
- we input multiple things, and make multiple predictions from it
- the input and output size do not need to be the same length

<div>
<img src="https://github.com/merriekay/S23-CS167-Notes/blob/main/images/day23_many2many.png?raw=1" width=400/>
</div>

Machine Translation:
- translating from one language to another

"Hello my name is" --> "Hola me llamo"

<div>
<img src="https://github.com/merriekay/S23-CS167-Notes/blob/main/images/day23_machine_translation.png?raw=1" width=400/>
</div>

## Many to Many (same size):

Can anyone think of an example of a ML model that is __many to many__ with the same size input/output?

<div>
<img src="https://github.com/merriekay/S23-CS167-Notes/blob/main/images/day23_many2many2.png?raw=1" width=200/>
</div>

Sign-Language Detector:
    
<div>
<img src="https://github.com/merriekay/S23-CS167-Notes/blob/main/images/day23_asl.png?raw=1" width=600/>
</div>

[[src](https://techcommunity.microsoft.com/t5/microsoft-teams-blog/introducing-sign-language-view-for-teams-meetings/ba-p/3671257)]

# Recurrent Neural Networks

__Recurrent Neural Networks__, also known as RNNs, are a class of neural networks that _allow previous outputs to be used as inputs_ while having hidden states.
- RNNs are really good at processing sequential data.
- RNNs can model all of the different configurations of input/output that we just went through
- RNNs must use __context__ when making a prediction.

# RNN

RNNs are really good at modeling __sequential data__--that is, data where the current observation depends on previous observations, and thus observations are not independent from each other.

Traditional neural networks view each observation as independent as the networks are not able to retain past or historical information. They have no memory of what happened in the past. 

RNNs achieve a sense of _memory_ by having a feedback loop. The feedback loop allows information to be passed within a layer, rather than only forward (as a feedforward neural network does). 

## RNNs have loops.
- Input $X_t$
- Output $h_t$
- Loop allows information to be passed from one step of the network to the next.

<div>
<img src="https://github.com/merriekay/S23-CS167-Notes/blob/main/images/day23_rnn.png?raw=1" width=700/>
</div>

## An Example:

Sometimes we only need to look at recent information to make a prediction:

<div>
<img src="https://github.com/merriekay/S23-CS167-Notes/blob/main/images/day23_rnn_time.png?raw=1" width=600/>
</div>

Consider a langauge model trying to predict the next word based on the previous ones. Let's predict the last word to this sequence:
- `"The clouds are in the _______"`

We don't need much extra context to make a prediction. However, 
- `"I grew up in France, I speak fluent _____"`

We can guess from recent information that it will be the name of a language, however, we need the context of France to make a prediction.

# Three main types of RNN:

There are 3 main types of RNN that we'll breifly discuss:
1. Traditiona Recurrent Neural Networks (RNN)
2. Long Short Term Momry Recrurent Neural Network (LSTM)
3. Gated Recurrent Unit Recurrent NEural Network

# Traditional Recurrent Neural Network:

Each cell has 2 inputs:
- the past and the present
- the output from the previous cell $h_{t-1}$, and the current input into the cell $X_t$.

<div>
<img src="https://github.com/merriekay/S23-CS167-Notes/blob/main/images/day23_unfolded_rnn.png?raw=1" width=800/>
</div>

[[src](https://towardsdatascience.com/a-brief-introduction-to-recurrent-neural-networks-638f64a61ff4)]

This is an image of the unfolded RNN to help understand what is going on. The length of the cell is equal to the number of time steps of the input sequence. 

In each cell the input of the current time step $x_t$, the hidden state of the previous cell $h_{t-1}$, and a bias are combined and then put through a $tanh$ activation function like so:

$$h_t = tanh( W_x x_t + W_h h_{t-1} + b) $$

Where $W_x$ and $W_h$ are weights.

## Advantages of Traditional RNNs

Due to their short term memory, RNNs can handle sequential data and identify patterns in historical data.

They can also handle inputs of varying lengths.

## Disadvantages of RNNs

The RNN suffers from the __vanishing gradient__ problem. 
- The graidents that are used to update the weights during backpropagation become very small
- Multiplying weights with a graident that is so close to zero prevents the network from learning new weights 
- This stops the learning and results in the RNN forgetting what is seen in longer sequences
- This problem gets worse with the more layers the network has.

# Long-Short-Term-Memory (LSTM)

LSTMs are a special type of RNN which tackle the __vanishing gradient__ problem. 

Essentially, the LSTM adds 3 gates:
- __forget gate__: decies how much long-term memory shall be kept
- __input gate__: decides which information shall be added to the long-term memory
- __output gate__: decides which parts of the cell state build the output, i.e. it's responsible for the short term memory

<div>
<img src="https://github.com/merriekay/S23-CS167-Notes/blob/main/images/day23_lstm.png?raw=1" width=800/>
</div>

## Advantages of  LSTMs

Can capture both the short and long term patterns of a seuqnce.

These are some of the most used RNNs.


## Disadvantages of LSTMs

Because LSTMs add complexity, they also are computationally more expensive, leading to longer training times. 

# Gated Recurrent Unit (GRU)

Similar to LSTMs, the GRU solves the vanishing gradient problem, but they do so using fewer gates--making them effective, and fast.

GRUs have two gates:
- a __reset gate__ which is responsible for the short-term memory as it decides how much past information is kept and disregarded
- an __update gate__ which is responsible for the long-term memory and is comparable to the LSTMs forget gate.
<div>
<img src="https://github.com/merriekay/S23-CS167-Notes/blob/main/images/day23_gru.png?raw=1" width=800/>
</div>

## Advantages of  GRUs

Solve vanishing gradient problem

Less computationally expensive than LSTMs, which makes them faster to train.

## Disadvantages of GRUs

GRUs do not have a separate hidden and cell state, so they might not be able to consider observations as far into the past as the LSTM.

# Project #2:

Very similar to Project #1, but I expect you to use some of the methods that we've talked about since project #1:
- PCA
- SVM
- Perceptron
- MLP
- Neural Networks
- Convolutional Neural Networks
- Recurrent Neural Networks

# Project #2 Description:

1. __Problem__: be sure to tell me what kind of machine learning is going on, what your target variable is, what your predictor variables are. What are you going to test in this experiment? What are your hypotheses (e.g. a model with 5 layers will perform better than a model with 3 layers)
2. __Data Preparation__: explain your data prep... what needed to be done to get your data in shape for your experiments?
3. __Research__: your code/experiments go here, be sure to include a couple graphs
4. __Analysis__: what did you discover? How did your hypotheses fare? What insights/recommendations do you have? What did you find that was interesting? Which model was your bets model, which models didn't work well? Why do you think this is? In general, describe your experiment, the results, and what they mean.
5. __Bumps in the Road__: What challenges did you encouter? How did you overcome these challenges?

## Getting Started:

Look for a dataset, use a different one than you used in Project #1. 

Think about what kinds of hypotheses you might test, and what kinds of models you think might perform well. 

I'm not giving you a chart to fill in like last time--which models you use are up to you. 

### Use the rest of class time to get started on this. 